In [1]:
import pandas
import psycopg2

from contextlib import closing

In [2]:
config = dict(dbname='art', user='art')

### Будем считать, что пользовательская сессия заканчивается, например, после `N` минут отсутствия активности

Вспомогательные функции

In [3]:
def execute(sql):
    with closing(psycopg2.connect(**config)) as conn:
        with conn.cursor() as cursor:
            cursor.execute(sql)
            conn.commit()

In [4]:
def select(sql, args=None):
    with closing(psycopg2.connect(**config)) as conn:
        with conn.cursor() as cursor:
            cursor.execute(sql, args)
            print(pandas.DataFrame(cursor.fetchall(), columns=[column[0] for column in cursor.description]))

### Создадим таблицу `action_type` и наполним данными

In [5]:
create_table_action_type = """CREATE TABLE action_type (
                                id   SERIAL PRIMARY KEY, 
                                name TEXT NOT NULL UNIQUE
                                );"""

insert_action_names = """INSERT INTO action_type(name) 
                         VALUES ('Copy'), ('Paste'), ('Backspace'), ('Run');"""

get_action_names = """SELECT *
                      FROM action_type"""

In [6]:
execute(create_table_action_type)
execute(insert_action_names)
select(get_action_names)

   id       name
0   1       Copy
1   2      Paste
2   3  Backspace
3   4        Run


### Создадим таблицу `user_action` и наполним данными

In [7]:
create_table_user_action = """CREATE TABLE user_action(
                                id         SERIAL PRIMARY KEY, 
                                action_id  INT NOT NULL REFERENCES action_type, 
                                time_stamp TIMESTAMP NOT NULL, 
                                user_id    INT NOT NULL
                                );"""

insert_actions = """INSERT INTO user_action(action_id, time_stamp, user_id) 
                    VALUES (1, '2020.02.29 14:45:49', 1),
                           (2, '2020.02.29 14:56:53', 1),
                           (3, '2020.03.01 12:20:15', 2),
                           (3, '2020.03.01 12:49:15', 2),
                           (1, '2020.03.01 12:52:15', 2),
                           (4, '2020.03.01 13:40:55', 2);"""

get_actions = """SELECT *
                 FROM user_action"""

In [8]:
execute(create_table_user_action)
execute(insert_actions)
select(get_actions)

   id  action_id          time_stamp  user_id
0   1          1 2020-02-29 14:45:49        1
1   2          2 2020-02-29 14:56:53        1
2   3          3 2020-03-01 12:20:15        2
3   4          3 2020-03-01 12:49:15        2
4   5          1 2020-03-01 12:52:15        2
5   6          4 2020-03-01 13:40:55        2


### Тогда мы можем объединить активности пользователей в сессии с помощью следующего запроса

In [9]:
def get_sessions(minutes):
    sessions = """SELECT id, action_id, time_stamp, user_id, SUM(is_new_session) OVER (ORDER BY user_id, time_stamp) AS session_id
                  FROM (
                        SELECT *, CASE 
                                    WHEN last_event IS NULL 
                                    OR EXTRACT('EPOCH' FROM time_stamp) - EXTRACT('EPOCH' FROM last_event) > %s
                                    THEN 1 ELSE 0 
                                  END AS is_new_session
                        FROM (
                              SELECT *, LAG(time_stamp, 1) OVER (PARTITION BY user_id ORDER BY time_stamp) AS last_event
                              FROM user_action
                             ) AS previous_event
                       ) AS session_start;"""
    select(sessions, (minutes * 60,))

In [10]:
minutes = 30
get_sessions(minutes)

   id  action_id          time_stamp  user_id  session_id
0   1          1 2020-02-29 14:45:49        1           1
1   2          2 2020-02-29 14:56:53        1           1
2   3          3 2020-03-01 12:20:15        2           2
3   4          3 2020-03-01 12:49:15        2           2
4   5          1 2020-03-01 12:52:15        2           2
5   6          4 2020-03-01 13:40:55        2           3
